In [1]:
import librosa
import librosa.display
import IPython.display as ipd
import os,glob,pickle
import soundfile
import numpy as np
import glob
import resampy

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
def extract_feature(file_name,mfcc,chroma,mel):
    x,sample_rate=librosa.load(os.path.join(file_name),res_type='kaiser_fast')
    if chroma:
        stft=np.abs(librosa.stft(x))
    result=np.array([])
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=x,sr=sample_rate,n_mfcc=40).T,axis=0)
        result=np.hstack((result,mfccs))
    if chroma:
        chroma=np.mean(librosa.feature.chroma_stft(S=stft,sr=sample_rate).T,axis=0)
        reult=np.hstack((result,chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(y=x,sr=sample_rate).T,axis=0)
        result=np.hstack((result,mel))
    return result

EMOTIONS IN THE RAVDESS AND TESS DATASET

In [3]:
emotions={
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised'
}

observed_emotions=['neutral','calm','happy',
                   'sad','angry','fearful','disgust','surprised']

LOAD THE DATA AND EXTRACT FEATURES FOR EACH SOUND FILE

In [4]:
import os
import glob
import numpy as np
from sklearn.model_selection import train_test_split



def load_data(test_size=0.2):
    x, y = [], []
    for file in glob.glob(r'C:\Users\91823\Desktop\New folder (2)\audio_speech_actors_01-24\Actor_*\*.wav'):
        file_name = os.path.basename(file)
        emotion=emotions[file_name.split("-")[0]]
       
        if emotion not in observed_emotions:
            continue

        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)

    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)


In [5]:
#split the dataset
import time
x_train,x_test,y_train,y_test=load_data(test_size=0.2)


In [ ]:
#glob.glob(r'C:\Users\91823\Desktop\New folder (2)\audio_speech_actors_01-24\Actor_*\*.wav')

In [6]:
print((x_train.shape[0],x_test.shape[0]))

(1152, 288)


In [7]:
print(f'features extracted:{x_train.shape[1]}')

features extracted:168


In [10]:
#initiate the multi layer perceptron classifier 
model=MLPClassifier(alpha=0.01,batch_size=256,epsilon=1e-08,hidden_layer_sizes=(300),learning_rate='adaptive',max_iter=500)

In [13]:
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
accuracy=accuracy_score(y_true=y_test,y_pred=y_pred)
print("Accuracy:{:.2f}%".format(accuracy*100))

Accuracy:100.00%


In [14]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       happy       1.00      1.00      1.00       288

    accuracy                           1.00       288
   macro avg       1.00      1.00      1.00       288
weighted avg       1.00      1.00      1.00       288



In [15]:
from sklearn.metrics import confusion_matrix
matrix=confusion_matrix(y_test,y_pred)
print(matrix)

[[288]]
